# web-scraping-project

Use the "Run" button to execute the code.

## Scraping quotes from BrainyQuote


Here are steps we follow:
outline
- we are going to scrape https://www.brainyquote.com/topics
- We'll get list of topic .For each topic we will get topic title ,topic page url and first quote about topic
- For each topic, we will we get all quotes in topic from topic page 
- For each quote we will get quote, quote url, author and author url
- For each topic we will create csv file


## Scrape list of topic BrainyQuote
- use requests to download page 
- user BS4 to parse and extract information 
- convert to pandas dataframe

In [11]:
! pip install requests --quiet

In [12]:
import requests

In [13]:
! pip install beautifulsoup4

In [14]:
from bs4 import BeautifulSoup

In [15]:
!pip install pandas --quiet

In [16]:
import pandas as pd

In [25]:
base_url='https://www.brainyquote.com/topics'
response = requests.get(base_url)

In [26]:
soup = BeautifulSoup(response.text, 'html.parser')

In [27]:
topic_class='topicContentName'

In [28]:
topic_name=soup.find_all('span', class_= topic_class)

In [29]:
topic_link_class='topicIndexChicklet bq_on_link_cl'

In [30]:
topic_link=soup.find_all('a', class_=topic_link_class)

In [31]:
def get_topic_page():
    base_url='https://www.brainyquote.com/topics'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
 

In [32]:
soup=get_topic_page()

In [33]:
soup.find('a')

<a aria-label="Search" class="bq-tnav-btn bq-nav-search-icon" href="/search"><img alt="Search" aria-hidden="true" class="s-btn bq-fa-invert" src="/st/img/4724283/fa/search.svg"/></a>

lets create some helper function

In [34]:
def scrape_topic_name(soup):
    topic_class='topicContentName'
    topic_name=soup.find_all('span', class_= topic_class)
    topic_list=[]
    for name in topic_name:
        topic_list.append(name.text.strip())
    return topic_list




scrape_topic_name can be used to get list of topic names

In [35]:
names=scrape_topic_name(soup)

In [36]:
names[:5]

['Age', 'Alone', 'Amazing', 'Anger', 'Anniversary']

In [37]:
def scrape_topic_url(soup):
    topic_link_class='topicIndexChicklet bq_on_link_cl'
    topic_link=soup.find_all('a', class_= topic_link_class)
    url_list=[]
    for link in topic_link:
        url_list.append('https://www.brainyquote.com'+link['href'])
    return url_list    
        


Similarly we have define function for url and quote 

In [38]:
urls=scrape_topic_url(soup)

In [39]:
urls[0:5]

['https://www.brainyquote.com/topics/age-quotes',
 'https://www.brainyquote.com/topics/alone-quotes',
 'https://www.brainyquote.com/topics/amazing-quotes',
 'https://www.brainyquote.com/topics/anger-quotes',
 'https://www.brainyquote.com/topics/anniversary-quotes']

In [40]:
def scrape_topic_quote(topic_link):
    quote_list=[]
    for quote in topic_link:
        quote_url = requests.get('https://www.brainyquote.com'+quote['href'])
        quote_con=quote_url.text
        quote_soup = BeautifulSoup(quote_con, 'html.parser')
        quotes_text = quote_soup.find_all('a', class_='b-qt')
        quote_list.append(quotes_text[0].text.strip())
    return quote_list

In [41]:
quote = scrape_topic_quote(topic_link)

In [42]:
quote[:5]

["You can't help getting older, but you don't have to get old.",
 "I restore myself when I'm alone.",
 'It is amazing what you can accomplish if you do not care who gets the credit.',
 'Holding on to anger is like grasping a hot coal with the intent of throwing it at someone else; you are the one who gets burned.',
 'May you live all the days of your life.']

lets pull all together

In [43]:

        
def scrape_topic():
    movie_url='https://www.brainyquote.com/topics'
    response = requests.get(movie_url)
 
    topic_dict={
       'name':scrape_topic_name(soup),
       'url':scrape_topic_url(soup),
       'quote':scrape_topic_quote(topic_link)
    }
    return pd.DataFrame(topic_dict)


In [44]:
scrape_topic()

,name,url,quote
0,Age,https://www.brainyquote.com/topics/age-quotes,"You can't help getting older, but you don't ha..."
1,Alone,https://www.brainyquote.com/topics/alone-quotes,I restore myself when I'm alone.
2,Amazing,https://www.brainyquote.com/topics/amazing-quotes,It is amazing what you can accomplish if you d...
3,Anger,https://www.brainyquote.com/topics/anger-quotes,Holding on to anger is like grasping a hot coa...
4,Anniversary,https://www.brainyquote.com/topics/anniversary...,May you live all the days of your life.
...,...,...,...
120,War,https://www.brainyquote.com/topics/war-quotes,"Know thy self, know thy enemy. A thousand batt..."
121,Wedding,https://www.brainyquote.com/topics/wedding-quotes,My father always wanted to be the corpse at ev...
122,Wisdom,https://www.brainyquote.com/topics/wisdom-quotes,The journey of a thousand miles begins with on...
123,Women,https://www.brainyquote.com/topics/women-quotes,A woman is like a tea bag - you can't tell how...


In [62]:
topic_csv=scrape_topic()
topic_csv.to_csv('topic.csv', index=None)

## Get the quote from the topic page 

- We are going to make function to get quote , quote url , author and author url for each topic

In [47]:
def get_repo_info(quotes, author):
    quote=quotes.text.strip()
    quote_url='https://www.brainyquote.com'+quotes['href']
    name=author.text.strip()
    name_url='https://www.brainyquote.com'+author['href']
    return quote,quote_url, name, name_url

## We are going to make a make faunction that take topic_url and create pandas datafarme

In [48]:
def get_repo_link(topic_url):
    quote_url = requests.get(topic_url)
    quote_con=quote_url.text
    quote_soup=BeautifulSoup(quote_con, 'html.parser')
    quotes_text = quote_soup.find_all('a', class_='b-qt')
    quote_author = quote_soup.find_all('a', class_='bq-aut')
    topic_repo_dict={
    'quote': [],
    'quote_url':[],
    'author':[],
    'author_url':[],
    }
    for i in range(len(quote_author)):
        repo_info=get_repo_info(quotes_text[i],quote_author[i])
        topic_repo_dict['quote'].append(repo_info[0])
        topic_repo_dict['quote_url'].append(repo_info[1])
        topic_repo_dict['author'].append(repo_info[2])
        topic_repo_dict['author_url'].append(repo_info[3])
    return pd.DataFrame(topic_repo_dict)
 

In [49]:
get_repo_link('https://www.brainyquote.com'+topic_link[1]['href'])

,quote,quote_url,author,author_url
0,I restore myself when I'm alone.,https://www.brainyquote.com/quotes/marilyn_mon...,Marilyn Monroe,https://www.brainyquote.com/authors/marilyn-mo...
1,Alone we can do so little; together we can do ...,https://www.brainyquote.com/quotes/helen_kelle...,Helen Keller,https://www.brainyquote.com/authors/helen-kell...
2,The time you feel lonely is the time you most ...,https://www.brainyquote.com/quotes/douglas_cou...,Douglas Coupland,https://www.brainyquote.com/authors/douglas-co...
3,The strongest man in the world is he who stand...,https://www.brainyquote.com/quotes/henrik_ibse...,Henrik Ibsen,https://www.brainyquote.com/authors/henrik-ibs...
4,"If you are lonely when you're alone, you are i...",https://www.brainyquote.com/quotes/jeanpaul_sa...,Jean-Paul Sartre,https://www.brainyquote.com/authors/jean-paul-...
5,Loneliness adds beauty to life. It puts a spec...,https://www.brainyquote.com/quotes/henry_rolli...,Henry Rollins,https://www.brainyquote.com/authors/henry-roll...
6,"It is far better to be alone, than to be in ba...",https://www.brainyquote.com/quotes/george_wash...,George Washington,https://www.brainyquote.com/authors/george-was...
7,Solitude vivifies; isolation kills.,https://www.brainyquote.com/quotes/joseph_roux...,Joseph Roux,https://www.brainyquote.com/authors/joseph-rou...
8,You only grow when you are alone.,https://www.brainyquote.com/quotes/paul_newman...,Paul Newman,https://www.brainyquote.com/authors/paul-newma...
9,"Let your mind alone, and see what happens.",https://www.brainyquote.com/quotes/virgil_thom...,Virgil Thomson,https://www.brainyquote.com/authors/virgil-tho...


In [63]:

for row in topic_link:
    pd_df=get_repo_link('https://www.brainyquote.com' +row['href'])
    pd_df.to_csv(row.text+'.csv', index=None)


In [64]:
import jovian

In [65]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vir14/web-scraping-project" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/vir14/web-scraping-project


'https://jovian.ai/vir14/web-scraping-project'